**Part 0:**

Done: readings and scaffolding code


**Part 1:**

Done: Create RNN class that extends from nn module

**Part 2:**

Done:Fill in the pieces for sampling text.

**Part 3:** 

Done: create an evaluation function/loop

**Part 4:**

To Do: Create a GRU Cell

I ran out of time - I created a class with an init and forward function but got stuck on figuring out the dimensions for the matrix multiplication.

**Part 5:**

Done: Run it and generate some text (using built-in GRU)

![alt text](https://nolans-cs-bucket.s3-us-west-1.amazonaws.com/ezgif.com-gif-maker(1).gif)

**Part 6:**

Done: Run it on a new dataset.  I trained on Emily Dickinson's poems and generated the following output. 

The network worked surprisingly well at picking up spelling, grammar, puncuation, and context. It however struggled in writing something longer than 200 characters, as it started repeating it's thoughts.

**Output from Network:**
'ng with despair. His head swam, but from the heat in his body he guessed that he had been given another draught.

An Ore stooped over him, and flung him some bread and a strip of raw dried flesh. Hes head swam, but from the heat in his body he guessed that he had been given another draught.

An Ore stooped over him, and flung him some bread and a strip of raw dried flesh. Hes head swam, but from the heat in his body he guessed that he had been given another draught.

An Ore stooped over him, and flung him some bread and a strip of raw dried flesh. Hes head swam, but from the heat in his body he guessed that he had been given another draught.

An Ore stooped over him, and flung him some bread and a strip of raw dried flesh. Hes head swam, but from the heat in his body he guessed that he had been given another draught.

An Ore stooped over him, and flung him some bread and a strip of raw dried flesh. Hes head swam, but from the heat in his body he guessed that he had been give'

In [0]:
# -*- coding: utf-8 -*-
"""Lab6.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1C7Kjvp4s0UiSrgbBhLYlhD8s5Iot59tv
"""
!pip3 install unidecode
#import subprocess
from PIL import Image, ImageDraw
import urllib.request
import os.path
from os import path
import tarfile
from torch.nn.parameter import Parameter
from PIL import Image, ImageDraw
from PIL import ImageFont
url='https://nolans-cs-bucket.s3-us-west-1.amazonaws.com/text.tar.gz'
location = './text.tar.gz'
if path.exists(location):
    print('already downloaded!')
else:
    urllib.request.urlretrieve(url,location )


data=tarfile.open(location)
data.extractall()
data.close()
#subprocess.run(['pip','install','unidecode'])
#! pip install unidecode
#! pip install torch

import unidecode
import string
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
 
 




all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt',encoding='utf8').read())
file_len = len(file)
print('file_len =', file_len)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  print(file[start_index:end_index])
  return file[start_index:end_index]

# Turn string into list of longs
def char_tensor(string):
  #new_string=''    
  #for i in range(len(string)):
      #if string[i]=='\n' and i<len(string) :
          #new_string=new_string+' '
      #else:
          #new_string=new_string+string[i]
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return Variable(tensor)

print(char_tensor('abcDEF'))

##the encoder

def make_strings(encoder,temperature):
    epochs_eval=100
    hidden_e = encoder.init_hidden()
    
    vecs=[]
    for i in range(epochs_eval):
        if i==0:
            output,hidden=encoder(char_tensor('a'),hidden_e)
        else:
            output,hidden=encoder(output_char2,hidden)
            output = output.squeeze(0).squeeze(0)
            output = torch.exp(output)
        #
        #output_char=torch.argmax(output,dim=0)
        output_char=torch.softmax(output/temperature,dim=0)
        output_char2= torch.multinomial(output_char,1).squeeze(0).squeeze(0)
        vecs.append(all_characters[output_char2.item()])
    return(vecs)

def evaluate_stuff(encoder,loss,epoch):
    with torch.no_grad():
       
        vecs1=make_strings(encoder,2)
        vecs2=make_strings(encoder,1) 
        vecs3=make_strings(encoder,0.5)
        
        
        string=''
        outstring1=string.join(vecs1)
        string1=outstring1[0:50]
        string2=outstring1[50:100]
        
        string=''
        outstring2=string.join(vecs2)
        string3=outstring2[0:50]
        string4=outstring2[50:100]
        
        string=''
        outstring3=string.join(vecs3)
        string5=outstring3[0:50]
        string6=outstring3[50:100]
        
         
        img = Image.new('RGB', (1200, 800), color = (255, 255, 255))
         
  
        fnt = ImageFont.truetype('arial.ttf', 40)
        d = ImageDraw.Draw(img)
        d.text((150, 50),'temperature : 2', font=fnt,fill=(0,0,0))
        d.text((150, 100),string1, font=fnt,fill=(0,0,0))
        d.text((150, 150),string2, font=fnt,fill=(0,0,0))
        d.text((150, 250),'temperature : 1', font=fnt,fill=(0,0,0))
        d.text((150, 300),string3, font=fnt,fill=(0,0,0))
        d.text((150, 350),string4, font=fnt,fill=(0,0,0))
        d.text((150, 450),'temperature : 0.5', font=fnt,fill=(0,0,0))
        d.text((150, 500),string5, font=fnt,fill=(0,0,0))
        d.text((150, 550),string6, font=fnt,fill=(0,0,0))
        d.text((150, 700),"epoch: " + str(epoch), font=fnt,fill=(0,0,0))
        d.text((700, 700),"loss: " + str(round(loss)), font=fnt,fill=(0,0,0))
        #img.save('C:/Users/Nolan/Documents/image_' + str(epoch) + '.png')


import torch.optim as optim

def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target
##returns two vectors, each 200 in length.These are the indices of the letters in the "all characters" variable, a random chunk from the data


class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    self.embedding = nn.Embedding(input_size,hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.net = nn.Sequential(nn.Linear(hidden_size,output_size))
  def forward(self, input_char, hidden):
    #pdb.set_trace()
    embedded = self.embedding(input_char).view(1,1,-1)
    output,hidden = self.gru(embedded,hidden)
    #linear layer
    linear_out=self.net(hidden)
    #softmax - spit out probabilities
    #output=torch.nn.functional.softmax(linear_out, dim=1)
    return linear_out,hidden
    
  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
  
##the decoder
    


encoder=RNN(100,100,100)
inp,target=random_training_set()
learning_rate=0.001
#mygru=myGRU(100,100)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




aws=0
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  encoder = nn.DataParallel(encoder,device_ids=[0,1,2,3])
  aws=1

if torch.cuda.is_available()==1:
  gpu=1
  workers=8
if torch.cuda.is_available()==0:
  gpu=0
  workers=2


if gpu==1:
	if aws==1:
		encoder.to(device)
	else:
		encoder.cuda()
else:
  print('no cuda')
#define the optimizer

encoder_optimizer = optim.Adam(encoder.parameters(),lr=1e-3)

objective = torch.nn.CrossEntropyLoss()



def train(inp, target,encoder,encoder_optimizer,objective,gpu):
  
  encoder_optimizer.zero_grad()
  input_length=inp.size(0)
  encoder_outputs=torch.zeros(chunk_len,100,device=device,requires_grad=True)
  
  hidden_e = encoder.init_hidden()
  hidden_e.require_grad=True
  
  #test=mygru(inp,hidden_e)
  encoder_output=hidden_e
  loss = 0.0
  loss = torch.tensor(loss,requires_grad=True)
  if gpu==1:
      inp=inp.cuda()
      encoder_output=encoder_output.cuda()
      target = target.cuda()
  
  for i in range(input_length-1):
    #pdb.set_trace()
    if i==0:
        encoder_hidden=hidden_e
    encoder_output,encoder_hidden=encoder(inp[i],encoder_hidden)
    encoder_outputs[i]=encoder_output[0,0]
    x=encoder_output.squeeze(0)
    test=inp[i+1].unsqueeze(0)
    loss=torch.sum(loss+objective(x,test))
  #decoder_input = torch.tensor([[0]],device=device)
  
  loss.backward()

  encoder_optimizer.step()
 
  return loss.item() 
  
 
epochs=1000
  # more stuff here...
loss_v=[]
for i in range(epochs):
    output=train(inp,target,encoder,encoder_optimizer,objective,gpu)
    loss_v.append(output)
    #if loss_v[-1]<1:
        #break
    #if i%10==0:
        #evaluate_stuff(encoder,output,i)
    print(output)

#with open('C:/Users/Nolan/Documents/RNN_model_2.txt', 'w') as f:
#   out=torch.save(encoder.state_dict(),'C:/Users/Nolan/Documents/RNN_model_2.txt')



In [0]:

encoder.eval()
temperature=2
epochs_eval=1000
hidden_e = encoder.init_hidden()

vecs=[]
for i in range(epochs_eval):
    if i==0:
        output,hidden=encoder(char_tensor('a'),hidden_e)
    else:
        output,hidden=encoder(output_char2,hidden)
    output = output.squeeze(0).squeeze(0)
    output = torch.exp(output)
    #
    #output_char=torch.argmax(output,dim=0)
    output_char=torch.softmax(output/temperature,dim=0)
    output_char2= torch.multinomial(output_char,1).squeeze(0).squeeze(0)
    vecs.append(all_characters[output_char2.item()])

string=''
outstring=string.join(vecs)
